In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("Churn_Modelling.csv")
X = df.iloc[:,3:-1].values
y = df.iloc[:,-1].values

FileNotFoundError: [Errno 2] No such file or directory: 'Churn_Modelling.csv'

In [ ]:
X

In [ ]:
#Label encoding gender
from sklearn.preprocessing import LabelEncoder
label_encoder_gender = LabelEncoder()
X[: ,2] = label_encoder_gender.fit_transform(X[: ,2])
print(X.shape)
print(X)

In [ ]:
#One hot Encoding the categorical variables countries
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers = [("encoder",OneHotEncoder(),[1])],remainder="passthrough")
X = np.array(ct.fit_transform(X))
X

In [ ]:
#splitting into training and test sets
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=0)
print(f"{X_train.shape} {X_test.shape}")


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)

X_test = sc.transform(X_test)

In [ ]:
print(X_test.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential



In [ ]:
ann_model = Sequential()
ann_model.add(tf.keras.layers.Dense(activation="relu",units=64,input_shape=(X_train.shape[1],)))
ann_model.add(tf.keras.layers.Dense(activation="relu",units=32))
ann_model.add(tf.keras.layers.Dense(activation="sigmoid",units=1))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
ann_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 64)                  │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
opt = Adam(learning_rate=0.01)
ann_model.compile(optimizer=opt,metrics=["accuracy"],loss="binary_crossentropy")

In [ ]:
#setup tensorboard and
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
import datetime
logs_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=logs_dir,histogram_freq=1)
earlystopping_callback = EarlyStopping(monitor="val_loss",patience=10,restore_best_weights=True)

In [ ]:
ann_model.fit(X_train,y_train,epochs=100,validation_data=(X_test,y_test),callbacks=[tensorboard_callback,earlystopping_callback])

In [ ]:
ann_model.save("Ann_Model.h5")

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit

In [ ]:
#saving the onehotencoder,labelencoder,standardscaler as pkl files
import pickle
with open("label_encoder_gender.pkl", "wb") as file:
  pickle.dump(label_encoder_gender,file)
with open("column_transformer.pkl", "wb") as file:
  pickle.dump(ct,file)
with open("standard_scaler.pkl", "wb") as file:
  pickle.dump(sc,file)

In [ ]:
#predicting new data
input_data = {
    'CreditScore' : 600,
     'Geography' : 'France',
    'Gender' : 'Male',
    'Age' : 40,
    'Tenure' : 3,
    'Balance' : 60000,
    'NumOfProducts' : 2,
    'HasCrCard' : 1,
    'IsActiveMember' : 1,
    'EstimatedSalary' : 50000
}
input_df = pd.DataFrame([input_data],)
input_df

In [ ]:
#load pkl files
with open("label_encoder_gender.pkl", "rb") as file:
  label_encoder = pickle.load(file)
with open("column_transformer.pkl", "rb") as file:
  c_transformer = pickle.load(file)
with open("standard_scaler.pkl", "rb") as file:
 stand_scaler= pickle.load(file)



In [ ]:
#Label encoding the gender from input
input_df['Gender'] = label_encoder.transform(input_df['Gender'])
input_df

In [46]:
#categorical encoding for Country column
transformed_df = c_transformer.transform(input_df.values)

In [47]:
scaled_df = stand_scaler.transform(transformed_df)
scaled_df

array([[ 0.98465111, -0.56811207, -0.57468161, -0.51937478,  0.91974271,
         0.11067641, -0.68335613, -0.24998937,  0.80424154,  0.64365658,
         0.96668786, -0.87258506]])

In [49]:
#predict the val
#load the model
from tensorflow.keras.models import load_model
model = load_model("Ann_Model.h5")
predicted_val = model.predict(scaled_df)
if predicted_val > 0.5:
  print("Customer is likely to stay")
else:
  print("Customer is likely to exit the bank!")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Customer is likely to exit the bank!
